In [27]:
import dash

from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

from dotenv import load_dotenv
import os

In [28]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [29]:
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage für die Stationen
select_value_query = """
    SELECT v.*, b.ort
    FROM value v
    JOIN box_id b ON v.senseid_fk = b.senseid;
"""
cursor.execute(select_value_query)
values = cursor.fetchall()

df = pd.DataFrame(values, columns=[desc[0] for desc in cursor.description])

cursor.close()
connection.close()

Verbindung zur Datenbank erfolgreich hergestellt.
           index                time  temperature  luftfeuchtigkeit  pm10  \
1629821  1724822 2024-06-17 15:30:05        22.78              0.00  0.00   
1629822  1724823 2024-06-17 15:30:06        25.95             40.68  2.60   
1629823  1724824 2024-06-17 15:40:01        25.30             88.90  0.63   
1629824  1724825 2024-06-17 15:40:02        25.79             57.58  1.63   
1629825  1724826 2024-06-17 15:40:02        26.90             53.90  1.20   
...          ...                 ...          ...               ...   ...   
1637282  1732283 2024-06-24 15:30:03        19.84             60.12  3.72   
1637283  1732284 2024-06-24 15:30:04        28.90             36.80  0.00   
1637284  1732285 2024-06-24 15:30:04        38.30             66.60  2.00   
1637285  1732286 2024-06-24 15:30:05        22.78              0.00  0.00   
1637286  1732287 2024-06-24 15:30:05        25.31             38.29  2.22   

         pm2_5           

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Werte Droppen

In [30]:
df = df[df['luftfeuchtigkeit'] != 100.00]
df = df[df['luftfeuchtigkeit'] != 99.90]
df = df[df["pm10"]<1200]
bedingung = df['temperature'] > df['temperature'].quantile(0.99)
df = df.drop(df[bedingung].index)

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [31]:
stations = df['ort'].unique()
stations_options = [{'label': 'Alle Stationen', 'value': 'all'}] + [{'label': station, 'value': station} for station in stations]

In [32]:
stations

array(['Hamburg', 'Mersch', 'Ettlingen', 'Stuttgart', 'München', 'Berlin',
       'Dortmund', 'Jena'], dtype=object)

In [33]:

app = dash.Dash(__name__)

# Layout der Dash-App definieren
app.layout = html.Div(id='app-container', children=[
    html.H1(
        children='Umweltüberwachung Dashboard',
        id='title',
    ),

    html.Button(
        'Toggle Dark/Light Mode',
        id='toggle-button',
        n_clicks=0,
        style={'margin-bottom': '20px'}
    ),

    html.Div(
        children='Wählen Sie eine Station und eine Kennzahl aus, um die Daten anzuzeigen.',
        id='description',
    ),

    dcc.Dropdown(
        id='station-dropdown',
        options=stations_options,
        value='all',
    ),

    html.Div(
        children='Wählen Sie eine Kennzahl:',
        id='metric-label',
        style={'marginTop': 20}
    ),
    
    dcc.RadioItems(
        id='metric-radio',
        options=[
            {'label': 'Durchschnitt', 'value': 'mean'},
            {'label': 'Median', 'value': 'median'},
            {'label': 'Varianz', 'value': 'var'},
            {'label': 'Standardabweichung', 'value': 'std'},
            {'label': 'Min', 'value': 'min'},
            {'label': 'Max', 'value': 'max'},
            {'label': 'Boxplot', 'value': 'boxplot'}
        ],
        value='mean',
        labelStyle={'display': 'inline-block', 'marginRight': 10}
    ),

    html.Div(
        children='Wählen Sie die Aggregationsstufe:',
        id='aggregation-label',
        style={'marginTop': 20}
    ),
    
    dcc.Checklist(
        id='aggregation-checklist',
        options=[
            {'label': 'Jahr', 'value': 'year'},
            {'label': 'Monat', 'value': 'month'},
            {'label': 'Tag', 'value': 'day'},
            {'label': 'Stunde', 'value': 'hour'}
        ],
        value=['year', 'month'],
        labelStyle={'display': 'inline-block', 'marginRight': 10}
    ),

    html.Div(
        dcc.Graph(id='temperature-graph'),
    ),
    html.Div(
        dcc.Graph(id='luftfeuchtigkeit-graph'),
    ),
    html.Div(
        dcc.Graph(id='pm-graph'),
    ),

    html.Div(
        children='Anzeigen der letzten 7 Tage:',
        id='last-7-days-label',
        style={'marginTop': 20}
    ),

    dcc.Checklist(
        id='last-7-days-checklist',
        options=[
            {'label': 'Letzte 7 Tage', 'value': 'last_7_days'}
        ],
        value=[],
        labelStyle={'display': 'inline-block', 'marginRight': 10}
    ),
])

# Funktion zur Berechnung von IQR
def calculate_metrics(group, metric):
    if metric == 'iqr':
        return group.quantile(0.75) - group.quantile(0.25)
    else:
        return group.agg(metric)

# Callback zur Aktualisierung der Diagramme
@app.callback(
    [Output('temperature-graph', 'figure'),
     Output('luftfeuchtigkeit-graph', 'figure'),
     Output('pm-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('metric-radio', 'value'),
     Input('aggregation-checklist', 'value'),
     Input('last-7-days-checklist', 'value'),
     Input('toggle-button', 'n_clicks')]
)
def update_graphs(selected_station, selected_metric, selected_aggregation, last_7_days, n_clicks):
    # Filtere den DataFrame nach der ausgewählten Station
    if selected_station != 'all':
        filtered_df = df[df['ort'] == selected_station]
    else:
        filtered_df = df.copy()

    # Bereinige den DataFrame von NaN-Werten
    filtered_df = filtered_df.dropna(subset=['temperature', 'luftfeuchtigkeit', 'pm2_5', 'pm10'])

    # Filtere die Daten der letzten 7 Tage, falls ausgewählt
    if 'last_7_days' in last_7_days:
        last_7_days_date = filtered_df['time'].max() - pd.Timedelta(days=7)
        filtered_df = filtered_df[filtered_df['time'] >= last_7_days_date]
        

        # Erstelle Diagramme ohne Aggregation
        temp_fig = px.line(filtered_df, x='time', y='temperature', title='Temperatur der letzten 7 Tage')
        temp_fig.update_traces(line=dict(color='red'))  # Linie rot machen
        humidity_fig = px.line(filtered_df, x='time', y='luftfeuchtigkeit', title='Luftfeuchtigkeit der letzten 7 Tage')
        humidity_fig.update_traces(line=dict(color='red'))  # Linie rot machen
        pm_fig = px.line(filtered_df, x='time', y=['pm2_5', 'pm10'], title='PM2.5 und PM10 der letzten 7 Tage')
        pm_fig.for_each_trace(lambda trace: trace.update(line=dict(color='blue')) if trace.name == 'pm2_5' else trace.update(line=dict(color='red')))  # PM2.5 Linie blau, andere rot

    else:
        # Überprüfen der gültigen Aggregationskombinationen
        if 'hour' in selected_aggregation and 'day' not in selected_aggregation:
            selected_aggregation.append('day')
        if 'day' in selected_aggregation and 'month' not in selected_aggregation:
            selected_aggregation.append('month')
        if 'month' in selected_aggregation and 'year' not in selected_aggregation:
            selected_aggregation.append('year')

        # Erstelle die Aggregationsspalte basierend auf der Auswahl
        aggregation = ''
        if 'year' in selected_aggregation:
            aggregation += '%Y'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

        if 'month' in selected_aggregation:
            aggregation += '-%m'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
            filtered_df['aggregation'] = pd.to_datetime(filtered_df['aggregation'], format='%Y-%m')
            filtered_df['aggregation'] = filtered_df['aggregation'].dt.strftime('%b. %Y')

        if 'day' in selected_aggregation:
            aggregation += '-%d'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

        if 'hour' in selected_aggregation:
            aggregation += ' %H'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
        if aggregation == '-%d':
            filtered_df['aggregation'] = filtered_df['time'].dt.day_of_year

        # Erstelle eine Liste aller möglichen Monate innerhalb des Datenbereichs
        all_months = pd.date_range(start=filtered_df['time'].min().replace(day=1), end=filtered_df['time'].max(), freq='MS').strftime('%b. %Y')

        if selected_metric == 'boxplot':
            # Boxplot-Diagramme
            temp_fig = px.box(filtered_df, x='aggregation', y='temperature', title='Boxplot Temperatur')
            humidity_fig = px.box(filtered_df, x='aggregation', y='luftfeuchtigkeit', title='Boxplot Luftfeuchtigkeit')
            pm_fig = px.box(filtered_df, x='aggregation', y=['pm2_5', 'pm10'], title='Boxplot PM2.5 und PM10')
        else:
            # Berechne die Kennzahlen
            daily_df = filtered_df.groupby('aggregation').agg({
                'temperature': selected_metric,
                'luftfeuchtigkeit': selected_metric,
                'pm2_5': selected_metric,
                'pm10': selected_metric
            }).reset_index()

            # Füge fehlende Monate hinzu
            missing_months = pd.DataFrame({'aggregation': all_months})
            daily_df = pd.merge(missing_months, daily_df, on='aggregation', how='left').fillna(0)

            # Benenne die Aggregationsachse basierend auf den ausgewählten Aggregationsstufen
            agg_label = '-'.join(selected_aggregation).capitalize()

            # Erstelle Diagramme
            temp_fig = px.line(daily_df, x='aggregation', y='temperature', title=f'{selected_metric.capitalize()} Temperatur')
            temp_fig.update_xaxes(title_text=agg_label)
            temp_fig.update_traces(line=dict(color='red'))  # Linie rot machen
            humidity_fig = px.line(daily_df, x='aggregation', y='luftfeuchtigkeit', title=f'{selected_metric.capitalize()} Luftfeuchtigkeit')
            humidity_fig.update_xaxes(title_text=agg_label)
            humidity_fig.update_traces(line=dict(color='red'))  # Linie rot machen
            pm_fig = px.line(daily_df, x='aggregation', y=['pm2_5', 'pm10'], title=f'{selected_metric.capitalize()} PM2.5 und PM10')
            pm_fig.update_xaxes(title_text=agg_label)
            pm_fig.for_each_trace(lambda trace: trace.update(line=dict(color='blue')) if trace.name == 'pm2_5' else trace.update(line=dict(color='red')))  # PM2.5 Linie blau, andere rot

    # Aktualisieren der Layouts basierend auf dem Dark/Light Mode
    if n_clicks % 2 == 1:
        graph_layout = dict(plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'))
    else:
        graph_layout = dict(plot_bgcolor='white', paper_bgcolor='white', font=dict(color='black'))

    temp_fig.update_layout(graph_layout)
    humidity_fig.update_layout(graph_layout)
    pm_fig.update_layout(graph_layout)

    return temp_fig, humidity_fig, pm_fig

# Callback zur Umschaltung des Modus
@app.callback(
    [Output('title', 'style'),
     Output('toggle-button', 'style'),
     Output('description', 'style'),
     Output('metric-label', 'style'),
     Output('aggregation-label', 'style'),
     Output('last-7-days-label', 'style'),
     Output('station-dropdown', 'style'),
     Output('metric-radio', 'labelStyle'),
     Output('aggregation-checklist', 'labelStyle'),
     Output('last-7-days-checklist', 'labelStyle'),
     Output('app-container', 'style')],
    [Input('toggle-button', 'n_clicks')]
)
def toggle_dark_light_mode(n_clicks):
    if n_clicks % 2 == 0:
        dark_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        light_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        dropdown_style = {'backgroundColor': 'black', 'color': 'white'}
        container_style = {'backgroundColor': 'black'}
    else:
        dark_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        light_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        dropdown_style = {'backgroundColor': 'white', 'color': 'black'}
        container_style = {'backgroundColor': 'white'}

    return (dark_mode_style, light_mode_style, dark_mode_style, dark_mode_style,
            dark_mode_style, dark_mode_style, dropdown_style, light_mode_style, 
            light_mode_style, light_mode_style, container_style)

# App ausführen
if __name__ == '__main__':
    app.run_server(debug=True,host='127.0.0.1', port=8050)


c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anac